# Script for calculating probability of T2 Henzie:





In [105]:
import random
import itertools

# Ramp is denoted by color they produced and ends with "r" (except for Utopia)
# Example below: 3 green dorks + Wild growth, Ignoble + Birds, Deathrite, Elves of deep shadow, Utopia sprawl, Exploration, Burgeoninng.
ramp = 4*["Gr"] + 2*["RGBr"] + 1*["RGBdeathrite"] + ["Br"] + 1*["RGB_utopia"] + 1*["exploration"] + 1*["burgeoning"]
# Land designations: "G" is untapped green source. "RB" is untapped RB source. "F" is forest type.
# Note: I count Exotic orchard as a colorless land and Plaza of heroes as RB (since it can't cast T1 ramp)
monoLands = 5*["GF"] + 2*["G"] + 5*["B"] + 3*["R"]; 
dualLands = 4*["RB"] + 1*["GBF"] + 2*["GB"] + 3*["RG"] + 1*["RGB"]
fetches = 2*["RGF/RB"] + ["RB/GBF"] + ["RGF/GBF"] + ["RGF/RB/GBF"] #"RB/GB" designate a fetch that can find a RB or GB source.
tappedLands = 2*["GT"] + 1*["RT"] + 1*["BT"] + 1*["RGBT"]
untappedLands = monoLands + dualLands + fetches
lands = untappedLands + tappedLands
# Make deck as list
deck = ramp + lands + (99 - len(ramp) - len(lands))*[""]

In [106]:
# Function that checks if three sources s = [s[0], s[1], s[2]] can make all Jund colors.
def makesRGB(s):
    bool = any([all([c[0] in s[0], c[1] in s[1], c[2] in s[2]]) for c in itertools.permutations(["R", "G", "B"], 3)])
    return(bool)

# Returns true if the hand can produce T2 Henzie
def checkHand(hand):
    untappedLandsInHand = [land for land in hand if land in untappedLands]
    tappedLandsInHand = [land for land in hand if land in tappedLands]
    landsInHand = untappedLandsInHand + tappedLandsInHand
    rampInHand = [r for r in hand if r in ramp]
    
    # Check burgeoning and exploration first
    if ("exploration" in rampInHand or "burgeoning" in rampInHand) and len(landsInHand) > 2:
        isExp = "exploration" in rampInHand #Subtle difference: Exploration requires 2 untapped lands, Burgeoning 1.
        rampInHand.remove("exploration") if isExp else rampInHand.remove("burgeoning")
        # Check all combinations of 3 lands. 
        for l in itertools.permutations(landsInHand, 3):
            # If first land is a non-green fetchland, we must choose a green dual land.
            if "/" in l[0] and l[0].count("G") == 1: 
                l0 = [s for s in l[0].split("/") if "G" in s][0]
                l = (l0, l[1], l[2])
            # Check if we have RGB, have untapped G on T1 and have untapped T2 if exploration.
            if makesRGB(l) and ("G" in l[0]) and not ("T" in l[0]+isExp*l[2]):
                return(True)
               
    ## Check remaing ramp spells (dorks + enchantments)    
    # Check if we have at least two lands and one ramp in hand
    if len(untappedLandsInHand) < 2 or len(rampInHand) == 0:
        return(False)
    # Generate all permutations of land1, land2 and ramp.
    combinations = [(l[0], l[1], r) for l in itertools.permutations(untappedLandsInHand) for r in rampInHand]
    # For all combinations, check whether they give all colors on T2.
    for comb in combinations:
        firstLand = comb[0]
        # If first land is a non-green fetchland, we must choose a green dual land.
        if "/" in firstLand and firstLand.count("G") == 1:
            firstLand = [s for s in firstLand.split("/") if "G" in s][0]
        secondLand = comb[1]
        r = comb[2]
        # If the following failure cases are all False, we have T2 Henzie
        c1 = not "G" in firstLand; # Check if we can cast ramp
        c2 = "utopia" in r and not ("F" in firstLand) # Check if first land is forest if ramp is Utopia sprawl
        c3 = "deathrite" in r and not ("/" in firstLand+secondLand) # Check if we have a fetch with Deathrite shaman
        c4 = not makesRGB([firstLand, secondLand, r]) # check colors
        if not any([c1, c2, c3, c4]):
            return(True)
        
    return(False) # If all combinations of lands and ramp fails to produce T2 Henzie    

In [107]:
hands = 100000
cardsDrawn = 8
count = 0
for numHands in range(hands):
    hand = random.sample(deck, cardsDrawn)
    if checkHand(hand):
        count = count + 1
print("Probability of T2 Henzie: " + str(count / hands * 100)[:4]+ " %")

Probability of T2 Henzie: 31.1 %
